In [3]:
#need pip install spotipy to run this
import json
import spotipy
import csv
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import base64
import itertools
from requests import post, get
from config import cid, secret

['2016', '2017', '2018', '2019', '2020', '2022']
['https://open.spotify.com/playlist/37i9dQZF1DX8XZ6AUo9R4R?si=3c0783980c4845f7', 'https://open.spotify.com/playlist/37i9dQZF1DWTE7dVUebpUW?si=0e58db4c553a43c9', 'https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8?si=3b770ab0d4214b13', 'https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9?si=08712985b70e4e0d', 'https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=d2cd57cf194b48cb', 'https://open.spotify.com/playlist/37i9dQZF1DXbJMiQ53rTyJ']


In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [6]:
results = sp.search(q='artist:' + 'Drake', type='artist')

In [7]:
auth_string = cid + ":" + secret
auth_bytes = auth_string.encode('utf-8')
auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
url = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': 'Basic ' + auth_base64,
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {"grant_type": "client_credentials"}
result = post(url, headers=headers, data=data)
json_result = json.loads(result.content)
token = json_result['access_token']
print(token)

BQBmrJ0QGyKJpdYpcx6fZdGlvJjJMttqyz4rMTbL9UmFpBhxGV7XJMQUFQOBNqe5dB3dx3AqQGEQHk4Qmib1aj4_Hx0oOEze0XRrfML4KLUlS6OTLMv1


In [8]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=fa88101bc2b04d96"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [9]:
sp.playlist_tracks(playlist_URI)['items'][0]['track']['duration_ms']

200040

In [11]:
genres = ''
song_name = []
artist_name = []
artist_pop = []
genre_list = []
album = []
track_pop = []
years = []
links = []
song_years = []
song_duration = []
#playlist_df = pd.read_csv('Resources/Playlistlinks.csv')
with open('Resources/Playlistlinks.csv') as file_obj:
    reader_obj = csv.reader(file_obj)
    next(reader_obj)
    for row in reader_obj:
        years.append(row[0])
        links.append(row[1])
for (link, year) in zip(links, years):
    #URI
    playlist_link = link
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    #track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    print(f"Starting link {link}")
    for track in sp.playlist_tracks(playlist_URI)["items"]:
        track_uri = track["track"]["uri"]
        #print(track_uri)
        #track = sp.track(track_uri)
        #print(track)
        #break
        song_genre = sp.track(track_uri)
        #print(song_genre['genre'])

        #Track name
        song_name.append(track["track"]["name"])

        #Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        #print(f"Uri:{artist_uri}")
        artist_info = sp.artist(artist_uri)

        #Name, popularity, genre
        artist_name.append(track["track"]["artists"][0]["name"])
        artist_pop.append(artist_info["popularity"])
        artist_genres = artist_info["genres"]
        for genre in artist_genres:
            if genre == artist_genres[-1]:
                genres = genres + genre + ' '
            else: 
                genres = genres + genre + ', '
        genre_list.append(genres)
        genres = ''
        #Album
        album.append(track["track"]["album"]["name"])

        song_duration.append((track['track']['duration_ms']) / 60000)

        song_years.append(year)
    
        #Popularity of the track
        track_pop.append(track["track"]["popularity"])
    
print(f"Song name count: {len(song_name)}")
print(f"Artist name count:{len(artist_name)}")
print(f"Artist pop count:{len(artist_pop)}")
print(f"Artist genre count:{len(genre_list)}")
print(f"Artist album count:{len(album)}")
print(f"Track pop count:{len(track_pop)}")
print(f"Artist name count:{len(song_years)}")

Starting link https://open.spotify.com/playlist/37i9dQZF1DX8XZ6AUo9R4R?si=3c0783980c4845f7
Starting link https://open.spotify.com/playlist/37i9dQZF1DWTE7dVUebpUW?si=0e58db4c553a43c9
Starting link https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8?si=3b770ab0d4214b13
Starting link https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9?si=08712985b70e4e0d
Starting link https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=d2cd57cf194b48cb
Starting link https://open.spotify.com/playlist/37i9dQZF1DXbJMiQ53rTyJ
Song name count: 500
Artist name count:500
Artist pop count:500
Artist genre count:500
Artist album count:500
Track pop count:500
Artist name count:500


In [14]:
top_fifty_df = pd.DataFrame({"Year": song_years,
                            "Song Name": song_name, 
                            "Artist Name": artist_name,
                            "Artist Popularity": artist_pop,
                            "Song Genre": genre_list,
                            "Album": album,
                            "Song Popularity": track_pop})
top_fifty_df

,Year,Song Name,Artist Name,Artist Popularity,Song Genre,Album,Song Popularity
0,2016,Starboy,The Weeknd,97,"canadian contemporary r&b, canadian pop, pop",Starboy,93
1,2016,One Dance,Drake,97,"canadian hip hop, canadian pop, hip hop, pop, ...",Views,87
2,2016,Love Yourself,Justin Bieber,91,"canadian pop, pop",Purpose (Deluxe),85
3,2016,Closer,The Chainsmokers,81,"dance pop, edm, electropop, pop, pop dance, tr...",Closer,85
4,2016,Hello,Adele,86,"british soul, pop, pop soul, uk pop",25,75
...,...,...,...,...,...,...,...
495,2022,Vegas (From the Original Motion Picture Soundt...,Doja Cat,87,"dance pop, pop",Vegas (From the Original Motion Picture Soundt...,84
496,2022,Ojitos Lindos,Bad Bunny,97,"reggaeton, trap latino, urbano latino",Un Verano Sin Ti,91
497,2022,Broadway Girls (feat. Morgan Wallen),Lil Durk,83,"chicago drill, chicago rap, drill, hip hop, ra...",7220 (Reloaded),73
498,2022,I Ain't Worried,OneRepublic,84,"dance pop, piano rock, pop",I Ain’t Worried (Music From The Motion Picture...,93


In [15]:
artist_name = 'Drake'
url = 'https://api.spotify.com/v1/search'
headers = {"Authorization": "Bearer " + token}
query = f"?q={artist_name}&type=artist&limit=1"
query_url = url + query
result = get(query_url, headers=headers)
json_result = json.loads(result.content)
json_result['artists']

{'href': 'https://api.spotify.com/v1/search?query=Drake&type=artist&offset=0&limit=1',
 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
   'followers': {'href': None, 'total': 74000074},
   'genres': ['canadian hip hop',
    'canadian pop',
    'hip hop',
    'pop',
    'rap',
    'toronto rap'],
   'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
   'id': '3TVXtAsR1Inumwj472S9r4',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb4293385d324db8558179afd9',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051744293385d324db8558179afd9',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1784293385d324db8558179afd9',
     'width': 160}],
   'name': 'Drake',
   'popularity': 97,
   'type': 'artist',
   'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
 'limit': 1,
 'next': 'https://api.spotify.com/v1/search?q

In [17]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2018', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=year%3A2018&limit=50&offset=1000&type=track:
 Bad request., reason: None